# Atencja — implementacja atencji między danym stanem dekodera a stanami kodera

Poniżej znajdują się dwie macierze `encoder_states` i `decoder_states` reprezentujące stan warstwy ukrytej po przetworzeniu każdego słowa przez koder oraz statyczne wektory zanurzeń związane z danym wejściem dekodera. Pojedynczy stan warstwy ukrytej zawiera zanurzenia o długości = 3, która to liczba jest równa rozmiarowi zanurzenia w dekoderze. W koderze mamy 4 stany warstw ukrytych, ponieważ przetwarzamy sekwencję składającą się z 4 tokenów.

W dekoderze znajduje się 5 tokenów, które są generowane na podstawie sekwencji przetwarzanej przez koder.

*Zadanie (1 punkt)*

Zadanie polega na: a) Obliczeniu podobieństwa wszystkich zanurzeń z dekodera (zapytań -- queries) do wszystkich zanurzeń kolejnych stanów kodera (kluczy -- keys) (pamiętaj, że macierze można transponować. W NumPy transponujemy macierz za pomocą `nazwa_macierzy. T`)

a) Softmax (zaimportowany z scipy) należy wykonać na utworzonej macierzy podobieństw. Uwaga: pamiętaj, aby zastosować softmax we właściwym wymiarze. Wszystkie ukryte stany kodera powinny być zmiękczone w kontekście danego stanu dekodera. W scipy funkcja softmax zawiera argument `axis`, który może pomóc.

b) Połącz macierz uwagi z kroku b) i „encoder_states”, aby wygenerować macierz zawierającą wektory kontekstu dla każdego tokena z dekodera.

In [1]:
import numpy as np
from scipy.special import softmax

# scipy.special.softmax(x, axis=None)

encoder_states = np.array(
    [[1.2, 3.4, 5.6],  # encoder's hidden layer output at the step 1, related to a given input token, e.g., I
     [-2.3, 0.2, 7.2],  # encoder's hidden layer output at the step 2, related to a given token, e.g., like
     [10.2, 0.2, 0.3],  # encoder's hidden layer output at the step 3, related to a given token, e.g., NLP
     [0.4, 0.7, 1.2]]  # encoder's hidden layer output at the step 4, related to a given token, e.g., "."
)

decoder_states = np.array(
    [[0.74, 0.23, 0.56],  # decoder's static word embedding at the step 1, related to a given token, e.g., <BOS>
     [7.23, 0.12, 0.55],  # decoder's static word embedding at the step 2, related to a given token, e.g., Ja
     [9.12, 4.23, 0.44],  # decoder's static word embedding at the step 3, related to a given token, e.g., lubię
     [4.1, 3.23, 0.5],  # decoder's static word embedding at the step 4, related to a given token, e.g., przetwarzanie
     [5.2, 3.1, 8.5]]  # decoder's static word embedding at the step 5, related to a given token, e.g., języka
)

similarities = np.dot(decoder_states, encoder_states.T)
attention_weights = softmax(similarities, axis=1)
context_vectors = np.dot(attention_weights, encoder_states)

print(similarities)
print(attention_weights)
print(context_vectors)

[[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]]
[[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]
[[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]


Oczekiwane wyjścia:

a) [[ 4.806 2.376 7.762 1.129] [ 12.164 -12.645 73.935 3.636] [ 27.79 -16.962 94.002 7.137] [ 18.702 -5.184 42.616 4.501] [ 64.38 49.86 56.21 14.45 ]]

b) [[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03] [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31] [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38] [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17] [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]

c) [[ 9.69108631 0.35799187 0.59163688] [10.2 0.2 0.3 ] [10.2 0.2 0.3 ] [10.2 0.2 0.3 ] [ 1.20254471 3.39909302 5.59850122]]


# Transformer
## Wykorzystanie modelu T5 opartego na transformerze do rozwiązywania różnych zadań NLP.

Dzisiaj poznamy nową bibliotekę — HuggingFace **transformers** (https://huggingface.co/docs/transformers/index) i użyjemy jej do rozwiązania kilku nieoczywistych problemów związanych z NLP za pomocą modelu **T5** .


HuggingFace transformers to jedna z najpopularniejszych bibliotek dostarczających nam wysokopoziomowe API do wykorzystania sieci neuronowych do rozwiązywania zadań związanych z przetwarzaniem języka naturalnego, przetwarzaniem dźwięku, wizją komputerową, a nawet scenariuszami multimodalnymi, w których musimy wykorzystywać wiele modalności naraz (np. odpowiadając na pytania o zdjęcia, wydobywając informacje z faktur).

Najpierw zainstalujmy zależności, samą bibliotekę `transformers` oraz moduł `sentencepiece`, który pomaga nam tokenizować dokumenty i przekształcać tokeny w kodowanie one-hot (ideę sentencepiece omówimy szczegółowo później).

**Ostrzeżenie**: jeśli zauważysz jakieś dziwne wyjątki, takie jak `cannot call from_pretrained na obiekcie None` gdzieś w twoim kodzie, zrestartuj środowisko używając: Runtime -> restart. Następnie uruchom komórki z kodem (bez ponownej instalacji bibliotek) jeszcze raz.

In [2]:
!pip install transformers
!pip install sentencepiece

Przeczytaj dokumentację dotyczącą modelu T5 dostępne tutaj: https://huggingface.co/docs/transformers/model_doc/t5

W sekcji `wnioskowanie` znajdziesz opis pokazujący w jaki sposób możemy pobrać wytrenowany model i użyć go do rozwiązania zadanego zadania. Po prostu użyj dostarczonego kodu, aby przetłumaczyć jakieś zdanie z angielskiego na niemiecki!

*(0.5 punkta)*

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=1024)

model = T5ForConditionalGeneration.from_pretrained('t5-base')

english_sentence = "This is a test sentence to be translated into German."

inputs = tokenizer.encode("translate English to German: " + english_sentence, return_tensors='pt')

outputs = model.generate(inputs, max_length=50, num_beams=4, early_stopping=True)
german_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(german_translation)

Dies ist ein Testsatz, der ins Deutsche übersetzt werden soll.


## Różne zadania

Eksperymentuj z innymi danymi wejściowymi, np. tymi, które przedstawiono na rysunku 1 przedstawionym w artykule przedstawiającym model T5 lub nawet szerszą listą przypadków użycia z Dodatku D dostarczonego z artykułem. Artykuł można znaleźć tutaj: https://arxiv.org/pdf/1910.10683.pdf

Uwaga: wśród dostarczonych danych wejściowych zastosowano kilka skrótów, niektóre z nich to:
- `stsb`: oznacza semantyczny test porównawczy podobieństwa tekstu. Biorąc pod uwagę dwa zdania, możemy obliczyć ich podobieństwa semantyczne, co może pomóc nam ustalić, czy jedno zdanie jest parafrazą drugiego.
- `cola`: oznacza Corpus of Linguistic Acceptability i pomaga nam określić, czy dane zdanie jest gramatyczne, czy niegramatyczne.

Jeśli spojrzysz na Dodatek D, skrótów jest więcej, są one związane z nazwami zadań przedstawionymi w benchmarku GLUE (dostępny tutaj: https://gluebenchmark.com/tasks) i benchmarku SUPERGLUE (dostępny tutaj: https:/ /super.gluebenchmark.com/tasks). Ideą GLUE i SUPERGLUE jest zebranie zestawu trudnych zadań, które można wykorzystać do oceny systemów wymagających rozumienia języka naturalnego.

**Wklej 3 przykładowe zadania i przetworzone dane wejściowe w komórce poniżej (1 punkt)**

In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Dane wejściowe dla zadania STSB
print("Zadanie STSB (Semantic Textual Similarity Benchmark):")

sentence1 = "The quick brown fox jumps over the lazy dog."
sentence2 = "A brown fox isn't quick, and the dog is lazy."

print(f"Zdanie 1: {sentence1}")
print(f"Zdanie 2: {sentence2}")

input_str = f"stsb sentence1: {sentence1} sentence2: {sentence2}"
input_ids = tokenizer.encode(input_str, return_tensors='pt')

# Generowanie odpowiedzi
outputs = model.generate(input_ids=input_ids,
                         max_length=128,
                         num_beams=4,
                         early_stopping=True)
similarity_score_str = tokenizer.decode(outputs[0], skip_special_tokens=True)
try:
    similarity_score = float(similarity_score_str)
    print(f"Podobieństwo semantyczne między zdaniem 1 i zdaniem 2 wynosi: {similarity_score}")
except ValueError:
    print("Nie udało się wyznaczyć podobieństwa semantycznego.")

# Dane wejściowe dla zadania CoLA
print("\nZadanie CoLA (Corpus of Linguistic Acceptability):")

sentence = "The cat is on the mat."

input_str = f"cola sentence: {sentence}"
input_ids = tokenizer.encode(input_str, return_tensors='pt')

print(sentence)

# Generowanie odpowiedzi
outputs = model.generate(input_ids=input_ids,
                         max_length=128,
                         num_beams=4,
                         early_stopping=True)
acceptability_label_str = tokenizer.decode(outputs[0], skip_special_tokens=True)

if acceptability_label_str.lower() == "unacceptable":
    print("Zdanie jest niepoprawne gramatycznie.")
elif acceptability_label_str.lower() == "acceptable":
    print("Zdanie jest poprawne gramatycznie.")
else:
    print("Nie udało się wyznaczyć poprawności gramatycznej zdania.")

# Dane wejściowe dla zadania MRPC
print("\nZadanie MRPC (Microsoft Research Paraphrase Corpus):")

sentence1 = "The company has been accused of fraudulent activity by the SEC."
sentence2 = "The SEC has made an accusation of fraudulent activity against the company."

print(f"Zdanie 1: {sentence1}")
print(f"Zdanie 2: {sentence2}")

input_str = f"mrpc sentence1: {sentence1} sentence2: {sentence2}"
input_ids = tokenizer.encode(input_str, return_tensors='pt')

# Generowanie odpowiedzi
outputs = model.generate(input_ids=input_ids,
                         max_length=128,
                         num_beams=4,
                         early_stopping=True)
inference_label_str = tokenizer.decode(outputs[0], skip_special_tokens=True)

if inference_label_str.lower() == "not_equivalent":
    print("Zdania różnią się w znaczeniu.")
elif inference_label_str.lower() == "equivalent":
    print("Zdania są podobne w znaczeniu.")
else:
    print("Nie udało się wyznaczyć podobieństwa semantycznego między zdaniami.")

Zadanie STSB (Semantic Textual Similarity Benchmark):
Zdanie 1: The quick brown fox jumps over the lazy dog.
Zdanie 2: A brown fox isn't quick, and the dog is lazy.
Podobieństwo semantyczne między zdaniem 1 i zdaniem 2 wynosi: 3.0

Zadanie CoLA (Corpus of Linguistic Acceptability):
The cat is on the mat.
Zdanie jest poprawne gramatycznie.

Zadanie MRPC (Microsoft Research Paraphrase Corpus):
Zdanie 1: The company has been accused of fraudulent activity by the SEC.
Zdanie 2: The SEC has made an accusation of fraudulent activity against the company.
Zdania są podobne w znaczeniu.


## Różne typy modeli

Dostępnych jest kilka modeli T5, które różnią się wielkością (i jakością). Im większy model, tym lepsze wyjście powinien generować. Eksperymentuj z niektórymi modelami z następującego zestawu:
- t5-small
- base t5
- t5-large
- t5-3b
- t5-11b

Sprawdź, czy można zaobserwować różnicę w jakości generowanych tekstów.

Porównaj również rozmiar modeli, możesz użyć funkcji `model.num_parameters()`, aby uzyskać numer parametru związany z każdym modelem. Dla każdego modelu, który jesteś w stanie załadować, podaj rozmiar w poniższej komórce (jeśli nie możesz załadować danego modelu, bo jest za duży, bez obaw, po prostu wpisz 'too big to load'). (*2 punkty*) 

In [2]:
# t5-small params number: 60506624
# t5-base params number: 222903552
# t5-large params number: 737668096
# t5-3b params number: 2851598336
# t5-11b params number: too big to load
from transformers import T5Tokenizer, T5ForConditionalGeneration

# t5-small
tokenizer = T5Tokenizer.from_pretrained('t5-small', model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained('t5-small')
print("t5-small params number: ", model.num_parameters())

# t5-base
tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained('t5-base')
print("t5-base params number: ", model.num_parameters())

# t5-large
tokenizer = T5Tokenizer.from_pretrained('t5-large', model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained('t5-large')
print("t5-large params number: ", model.num_parameters())

# t5-3b
tokenizer = T5Tokenizer.from_pretrained('t5-3b', model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained('t5-3b')
print("t5-3b params number: ", model.num_parameters())

# t5-11b
tokenizer = T5Tokenizer.from_pretrained('t5-11b', model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained('t5-11b')
print("t5-11b params number: ", model.num_parameters())


t5-small params number:  60506624
t5-base params number:  222903552
t5-large params number:  737668096


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16777216 bytes.

## T5 specyficzne dla języka (ZADANIE OPCJONALNE – nie musisz tutaj podawać kodu)

Istnieją nawet alternatywy dla oryginalnych modeli T5. Ponieważ model T5 był trenowany na języku angielskim, dostępne są modele specyficzne dla innych języków, np. polskiego (np. plT5 zaproponowany przez Allegro - https://huggingface.co/allegro/plt5-small). Polski model został przeszkolony do rozwiązywania zestawu zadań zebranych w benchmarku KLEJ, który stanowi polską analogię do benchmarku GLUE: https://klejbenchmark.com.

Więcej szczegółów na temat plT5 można znaleźć w artykule badawczym: https://arxiv.org/pdf/2205.08808.pdf. Tabela 2 przedstawia przykładowe podpowiedzi, które można wykorzystać do rozwiązania niektórych zadań wymienionych w KLEJ.

Możesz wyszukać alternatywę dla oryginalnego T5, na przykład tę związaną z Twoim językiem, i poeksperymentować z nią (**to zadanie nie jest obowiązkowe**).

In [6]:
# (OPTIONAL): If you want, experiment with some alternative models (like language-related, e.g., plT5 related to Polish)

## Flan-T5

Pod koniec 2022 roku zaproponowano ewolucję T5 o nazwie Flan-T5. Ten model jest również dostarczany przez bibliotekę transformatorów HuggingFace. Odwiedź tę stronę: https://huggingface.co/docs/transformers/model_doc/flan-t5, aby zobaczyć, jak możesz użyć tego modelu (wystarczy zmienić nazwę modelu, aby pobrać!).

Flan-T5 jest znacznie potężniejszy niż T5. Możesz zajrzeć do Dodatku D zawartego w artykule opisującym Flan T5, aby zapoznać się z niektórymi formatami wejściowymi (monitami) i generowanymi wartościami. Artykuł jest tutaj: https://arxiv.org/pdf/1910.10683.pdf. Powinieneś skupić się na polach „przetworzonych danych wejściowych”, ponieważ są to reprezentacje używane przez model. Eksperymentuj z wybranymi zadaniami i sprawdź, czy możesz uzyskać takie same wyniki! W poniższym kodzie wklej kod ładujący model Flan-T5 i wykorzystujący go do rozwiązania wybranych zadań. (*1 punkt*)

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')

# Dane wejściowe dla zadania STSB
print("Zadanie STSB (Semantic Textual Similarity Benchmark):")

sentence1 = "The quick brown fox jumps over the lazy dog."
sentence2 = "A brown fox isn't quick, and the dog is lazy."

print(f"Zdanie 1: {sentence1}")
print(f"Zdanie 2: {sentence2}")

input_str = f"stsb sentence1: {sentence1} sentence2: {sentence2}"
input_ids = tokenizer.encode(input_str, return_tensors='pt')

# Generowanie odpowiedzi
outputs = model.generate(input_ids=input_ids,
                         max_length=128,
                         num_beams=4,
                         early_stopping=True)
for i in range(0, len(outputs)):
    print(f"Odpowiedź {i}: {tokenizer.decode(outputs[i], skip_special_tokens=True)}")
similarity_score_str = tokenizer.decode(outputs[0], skip_special_tokens=True)
try:
    similarity_score = float(similarity_score_str)
    print(f"Podobieństwo semantyczne między zdaniem 1 i zdaniem 2 wynosi: {similarity_score}")
except ValueError:
    print("Nie udało się wyznaczyć podobieństwa semantycznego.")

# Dane wejściowe dla zadania CoLA
print("\nZadanie CoLA (Corpus of Linguistic Acceptability):")

sentence = "The cat is on the mat."

input_str = f"cola sentence: {sentence}"
input_ids = tokenizer.encode(input_str, return_tensors='pt')

print(sentence)

# Generowanie odpowiedzi
outputs = model.generate(input_ids=input_ids,
                         max_length=128,
                         num_beams=4,
                         early_stopping=True)
acceptability_label_str = tokenizer.decode(outputs[0], skip_special_tokens=True)

if acceptability_label_str.lower() == "unacceptable":
    print("Zdanie jest niepoprawne gramatycznie.")
elif acceptability_label_str.lower() == "acceptable":
    print("Zdanie jest poprawne gramatycznie.")
else:
    print("Nie udało się wyznaczyć poprawności gramatycznej zdania.")

# Dane wejściowe dla zadania MRPC
print("\nZadanie MRPC (Microsoft Research Paraphrase Corpus):")

sentence1 = "The company has been accused of fraudulent activity by the SEC."
sentence2 = "The SEC has made an accusation of fraudulent activity against the company."

print(f"Zdanie 1: {sentence1}")
print(f"Zdanie 2: {sentence2}")

input_str = f"mrpc sentence1: {sentence1} sentence2: {sentence2}"
input_ids = tokenizer.encode(input_str, return_tensors='pt')

# Generowanie odpowiedzi
outputs = model.generate(input_ids=input_ids,
                         max_length=128,
                         num_beams=4,
                         early_stopping=True)
inference_label_str = tokenizer.decode(outputs[0], skip_special_tokens=True)

if inference_label_str.lower() == "not_equivalent":
    print("Zdania różnią się w znaczeniu.")
elif inference_label_str.lower() == "equivalent":
    print("Zdania są podobne w znaczeniu.")
else:
    print("Nie udało się wyznaczyć podobieństwa semantycznego między zdaniami.")


Zadanie STSB (Semantic Textual Similarity Benchmark):
Zdanie 1: The quick brown fox jumps over the lazy dog.
Zdanie 2: A brown fox isn't quick, and the dog is lazy.
Odpowiedź 0: sentence1: The quick brown fox jumps over the lazy dog sentence2: A brown fox isn't quick and the dog is lazy
Nie udało się wyznaczyć podobieństwa semantycznego.

Zadanie CoLA (Corpus of Linguistic Acceptability):
The cat is on the mat.
Nie udało się wyznaczyć poprawności gramatycznej zdania.

Zadanie MRPC (Microsoft Research Paraphrase Corpus):
Zdanie 1: The company has been accused of fraudulent activity by the SEC.
Zdanie 2: The SEC has made an accusation of fraudulent activity against the company.
Nie udało się wyznaczyć podobieństwa semantycznego między zdaniami.


## (OPCJONALNIE) Dostrajanie

Możesz nawet dostroić model T5/Flan-T5, aby rozwiązać wybrane zadanie. Możesz załadować istniejący model T5/Flan-T5, który jest już przeszkolony do rozwiązywania niektórych zadań, i wykorzystać moc 'transfery wiedzy', aby nauczyć go rozwiązywać różne zadania. Jest to o wiele lepsze niż trenowanie sieci od zera i powinno wymagać mniejszej liczby przykładów szkoleniowych.

Faza dostrajania jest dość złożona. Jednak opis krok po kroku można znaleźć tutaj: https://www.philschmid.de/fine-tune-flan-t5

Możesz spróbować dostroić wybrany model.